## BERT

1. BERT의 pre-trained
- MLM(Masked Language Model): 전체 입력 토큰 중 15%를 masking하여 학습
- NSP(Next Sentence Prediction): 문장 A 다음에 문장 B가 오는 것이 적절한지 판단

2. BERT는 Token Embedding + Segment Embedding + Position Embeddings 3가지로 구성된다. 

3. BERT-base
  - L = 트랜스포머 블록 = 12
  - H = 히든 레이어 차원 수, 임베딩 차원 수 = 768
  - A = 셀프 어텐션 헤드 수 = 12
* Reference
https://www.notion.so/BERT-Pre-training-of-Deep-Bidirectional-Transformers-for-Language-Understanding-Devlin-et-al-201-1faa5a8dee4880b99f9bf8da91043353

In [1]:
# 오큰화, 위치 인코딩 -> 임베딩
import torch
import torch.nn as nn

text = "BERT는 양방향 이해를 통해 자연어 이해에서 일반적으로 디코더 모델 대비 높은 성능을 보인다. 입력에 대해 병렬 연산이 가능하므로 빠른 학습과 추론이 가능하다."

input_text_list = text.split()
print(f"input_text_list: {input_text_list}")

str2idx = {word:idx for idx, word in enumerate(input_text_list)}
idx2str = {idx:word for idx, word in enumerate(input_text_list)}

print(f"str2idx: {str2idx}")
print(f"idx2str: {idx2str}")

input_text_list: ['BERT는', '양방향', '이해를', '통해', '자연어', '이해에서', '일반적으로', '디코더', '모델', '대비', '높은', '성능을', '보인다.', '입력에', '대해', '병렬', '연산이', '가능하므로', '빠른', '학습과', '추론이', '가능하다.']
str2idx: {'BERT는': 0, '양방향': 1, '이해를': 2, '통해': 3, '자연어': 4, '이해에서': 5, '일반적으로': 6, '디코더': 7, '모델': 8, '대비': 9, '높은': 10, '성능을': 11, '보인다.': 12, '입력에': 13, '대해': 14, '병렬': 15, '연산이': 16, '가능하므로': 17, '빠른': 18, '학습과': 19, '추론이': 20, '가능하다.': 21}
idx2str: {0: 'BERT는', 1: '양방향', 2: '이해를', 3: '통해', 4: '자연어', 5: '이해에서', 6: '일반적으로', 7: '디코더', 8: '모델', 9: '대비', 10: '높은', 11: '성능을', 12: '보인다.', 13: '입력에', 14: '대해', 15: '병렬', 16: '연산이', 17: '가능하므로', 18: '빠른', 19: '학습과', 20: '추론이', 21: '가능하다.'}


In [5]:
input_ids = [str2idx[word] for word in input_text_list]
input_ids

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]

In [27]:
# 임베딩
# BERT의 임베딩 차원 (d_model, hidden)은 base 기준 768
embedding_dim = 768
max_position_embedding = 512

embed_layer = nn.Embedding(len(str2idx), embedding_dim)
position_embed_layer = nn.Embedding(max_position_embedding, embedding_dim)

position_ids = torch.arange(len(input_ids), dtype = torch.long).unsqueeze(0)
position_encodings = position_embed_layer(position_ids)

token_embeddings = embed_layer(torch.tensor(input_ids)).unsqueeze(0)

input_embeddings = token_embeddings + position_encodings
input_embeddings, input_embeddings.shape

(tensor([[[-1.0925,  1.5432,  0.7238,  ...,  0.8476, -2.7941,  0.1010],
          [ 1.1692,  0.4264, -2.9783,  ...,  2.1519,  0.5225, -1.2798],
          [-0.8316,  1.8479, -0.9720,  ..., -1.1317, -0.4781, -1.0713],
          ...,
          [ 1.3848,  0.5536, -0.9256,  ..., -0.3048, -3.1033, -0.7299],
          [-1.5822, -1.7568, -0.9015,  ..., -2.6239, -0.2345,  1.5677],
          [-0.3965,  2.4432, -0.6863,  ...,  1.9256,  0.9645, -0.0621]]],
        grad_fn=<AddBackward0>),
 torch.Size([1, 22, 768]))

## BERT Embedding layer

In [26]:
class BertEmbeddings(nn.Module):
  def __init__(self, vocab_size: int, embedding_dim: int = 768,
               max_position_embeddings: int = 512, type_vocab_size: int = 2,
               dropout_rate: float = 0.1):
    """
    Args:
        vocab_size (int): 어휘 집합 크기 
        embedding_dim (int): BERT-base의 임베딩 차원
        max_position_embeddings (int): 최대 시퀀스 길이 
        type_vocab_size (int): 세그먼트 타임 개수(문장 A/B 등)
        dropout_rate (float): 드롭 아웃
    """
    super().__init__()
    
    self.token_embeddings = nn.Embedding(vocab_size, embedding_dim)
    self.position_embeddings = nn.Embedding(max_position_embeddings, embedding_dim)
    # 세그먼트 임베딩 
    self.token_type_embedding = nn.Embedding(type_vocab_size, embedding_dim)
    self.dropout = nn.Dropout(dropout_rate)
    
    # 포지션 ID는 보통 [0, 1, 2, ..., seq_len-1] 형태로 미리 생성하여 재사용
    self.register_buffer('position_ids', torch.arange(max_position_embeddings)).unsqueeze(0)
    
  def forward(self, input_ids: torch.Tensor, token_type_ids: torch.Tensor = None) -> torch.Tensor:
    """
    BERT 모델의 입력 임베딩을 생성합니다.

    Args:
        input_ids (torch.Tensor): 입력 토큰 ID (batch_size, seq_len)
        token_type_ids (torch.Tensor, optional): 토큰 타입 ID (batch_size, seq_len).
                                                제공되지 않으면 모든 토큰을 0으로 간주.

    Returns:
        torch.Tensor: 결합된 임베딩 (batch_size, seq_len, embedding_dim)
    """
    seq_len = input_ids.size(1)
    batch_size = input_ids.size(0) # 보통 맨 앞을 배치 차원으로 위치시킴
    
    token_embeddings = self.token_embeddings(input_ids)
    # 현재 시퀀스에 맞는 position_ids만 사용 
    position_ids = self.register_buffer[:,:seq_len]
    position_embeddings = self.position_embeddings(position_ids)
    
    if token_type_ids is None:
      # input_ids와 동일한 device에 생성
      token_type_ids = torch.zeros((batch_size, seq_len), dtype = torch.long, device = input_ids.device)
    segment_embeddings = self.token_type_embedding(token_type_ids)
    
    input_embeddings = token_embeddings + position_embeddings + segment_embeddings
    
    input_embeddings = self.dropout(input_embeddings)
    
    return input_embeddings
        

In [28]:
## 어텐션
## 스케일 점곱

from math import sqrt
import torch.nn.functional as F
import math

def scaled_dot_product_attention(querys: torch.Tensor, 
                                 keys: torch.Tensor, 
                                 values: torch.Tensor, 
                                 attention_mask: torch.Tensor = None, # 패딩 마스크 
                                 is_casual: bool = False):
  dim_k = querys.size(-1)
  scores = querys @ keys.transpose(-2,-1) / sqrt(dim_k)
  
  if attention_mask is not None:
    # 마스크가 0인 위치는 -inf로 채워서 softmax시 0이 되도록 함
    scores = scores.masked_fill(attention_mask == 0, float('-inf'))
    
  attention_weights = F.softmax(scores, dim = -1)
  return torch.matmul(attention_weights, values)

In [29]:
## 멀티헤드 어텐션

class MultiHeadAttention(nn.Module):
  def __init__(self, d_model: int, nhead: int):
    super().__init__()
    # d_model(embedding_dim)은 nhead로 나누어 떨어져야됨 
    assert d_model % nhead == 0, "embedding_dim must be divisible bu nhead"
    
    self.dim_k = embedding_dim // nhead # 각 헤드의 차원
    self.nhead = nhead
    self.d_model = d_model
    
    self.weight_q = nn.Linear(d_model, d_model)
    self.weight_k = nn.Linear(d_model, d_model)
    self.weight_v = nn.Linear(d_model, d_model)
    
    self.concat_linear = nn.Linear(d_model, d_model) # 멀티헤드 어텐션이므로 모든 어텐션 헤드 출력 후 합쳐야됨 
    
  def forward(self, 
              querys: torch.Tensor,
              keys: torch.Tensor, 
              values: torch.Tensor,
              attention_mask: torch.Tensor = None) -> torch.Tensor:
    
    batch_size = querys.size(0)
    
    # Q, K, V를 Linear 레이어에 통과시킨 후, 여러 헤드로 분할 (view와 transpose 사용)
    # 예: (batch_size, seq_len, embedding_dim) -> (batch_size, seq_len, num_heads, d_k)
    # -> (batch_size, num_heads, seq_len, d_k)
    query = self.weight_q(query).view(batch_size, -1, self.nhead, self.dim_k).transpose(1, 2)
    key = self.weight_k(key).view(batch_size, -1, self.nhead, self.dim_k).transpose(1, 2)
    value = self.weight_v(value).view(batch_size, -1, self.nhead, self.dim_k).transpose(1, 2)
    
    attn_output, attn_weights = scaled_dot_product_attention(query, key, value, attention_mask=attention_mask, is_causal=False)
    attn_output = attn_output.transpose(1, 2).contiguous().view(batch_size, -1, self.d_model)

    output = self.concat_linear(attn_output)
    
    return output

In [30]:
class PositionwiseFeedForward(nn.Module):
    """
    BERT의 Position-wise Feed-Forward Network (FFN)를 구현합니다.
    (Pre-Layer Normalization 방식)
    """
    def __init__(self, d_model: int, dim_feedforward: int, dropout_rate: float):
        """
        Args:
            d_model (int): 모델의 임베딩 차원 (hidden_size).
            dim_feedforward (int): FFN의 중간 레이어 차원 (보통 d_model * 4).
            dropout_rate (float): 드롭아웃 비율.
        """
        super().__init__()
        
        self.linear1 = nn.Linear(d_model, dim_feedforward)
        self.linear2 = nn.Linear(dim_feedforward, d_model)
      
        self.activation = nn.GELU()
        
        self.dropout = nn.Dropout(dropout_rate)
        self.norm = nn.LayerNorm(d_model)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Args:
            x (torch.Tensor): FFN의 입력 텐서 (batch_size, seq_len, d_model).
        Returns:
            torch.Tensor: FFN의 출력 텐서 (batch_size, seq_len, d_model).
        """
        # 1. Layer Normalization (Pre-LN)
        # 먼저 층정규화를 함 
        norm_x = self.norm(x)

        # sublayer의 내부 처리
        ffn_output = self.linear1(norm_x)
        ffn_output = self.activation(ffn_output)
        ffn_output = self.dropout(ffn_output) # 첫 번째 드롭아웃 (FFN 내부)
      
        ffn_output = self.linear2(ffn_output)
        
        # 잔차 연결 (Residual Connection)
        ffn_output = self.dropout(ffn_output) # 두 번째 드롭아웃 (잔차 연결 전)
        output = x + ffn_output
        
        return output


In [31]:
class TransformerEncoder(nn.Module):
  def __init__(self, d_model, nhead, dim_feedforward, dropout):
    super().__init__()
    self.attn = MultiHeadAttention(d_model, d_model, nhead)
    self.norm1 = nn.LayerNorm(d_model) 
    self.dropout1 = nn.Dropout(dropout) # 여기서 dropout의 비율은 이미 정의됨 
    self.feed_forward = PositionwiseFeedForward(d_model, dim_feedforward, dropout)
  
  # 입력 src 
  def forward(self, src) -> torch.Tensor:
    norm_x = self.norm1(src)
    # query, key, value
    attn_output = self.attn(norm_x, norm_x, norm_x)
    # 잔차 연결, 입력 값 src에 어텐션 + 드롭아웃에 통과시킨 후 그 출력
    x = src + self.dropout1(attn_output) 
    
    # 피드 포워드
    x = self.feed_forward(x)
    return x